In [33]:
import pandas as pd
df = pd.read_stata('chainlinked_impoorts.dta')

In [35]:
df.to_csv("govdef_GDP1.csv", index = False)

In [2]:
print(df.loc[df['year'] == 1810, divide_cols])

'C:\\Users\\lehbib\\Documents\\GitHub\\Global-Macro-Database\\data\\final'

In [25]:
row = df.loc[(df['year'] == 1810)& (df['ISO3'] == "USA")]
print(row[['govdef_GDP', 'CS2_govdef_GDP']])

     govdef_GDP  CS2_govdef_GDP
865     0.20867        7.115948


In [17]:
print(df.loc[(df['year'] == 1810)& (df['ISO3'] == "USA"), divide_cols])

     ADB_govdef_GDP  AFDB_govdef_GDP  AMF_govdef_GDP  BCEAO_govdef_GDP  \
865             NaN              NaN             NaN               NaN   

     CEPAC_govdef_GDP  EUS_govdef_GDP  FRANC_ZONE_govdef_GDP  FZ_govdef_GDP  \
865               NaN             NaN                    NaN            NaN   

     IMF_FPP_govdef_GDP  IMF_GFS_govdef_GDP  IMF_WEO_govdef_GDP  \
865                 1.0                 NaN                 NaN   

     Mitchell_govdef_GDP  OECD_EO_govdef_GDP  CS2_govdef_GDP  CS1_govdef_GDP  
865            -0.978011                 NaN        7.115948             NaN  


In [29]:
import pandas as pd
df = pd.read_stata('chainlinked_govdef_GDP.dta')
df['check'] = ""
exclude = ['ISO3', 'year', 'chainlinking_ratio', 'source', 'source_change', 'check', 'govdef_GDP']
divide_cols = [col for col in df.columns if col not in exclude]
df[divide_cols] = df[divide_cols].div(df['govdef_GDP'], axis=0)
condition = (abs(df[divide_cols]) > 5) | (abs(df[divide_cols]) < 0.2)
df['check'] = condition.any(axis=1).astype(int)
df.loc[df['check'] == 1, ['ISO3','year', 'check']]

# Export df to csv 
df.to_csv('chainlinked_govdef_GDP.csv', index=False)

In [32]:
import pandas as pd

# -------------------------------------------------------------
# 1. Read a *fresh* copy of the file
# -------------------------------------------------------------
df = pd.read_stata("chainlinked_govdef_GDP.dta")

# -------------------------------------------------------------
# 2. Which columns should be divided by govdef_GDP?
#    Rule:  end with "_govdef_GDP"  but are *not* "govdef_GDP"
# -------------------------------------------------------------
ratio_cols = [
    c for c in df.columns if c.endswith("_govdef_GDP") and c != "govdef_GDP"
]

# -------------------------------------------------------------
# 3. Calculate the ratios in a **separate** DataFrame
# -------------------------------------------------------------
ratio = df[ratio_cols].div(df["govdef_GDP"], axis=0)

# -------------------------------------------------------------
# 4. Find the outliers
#    |x| < 0.2  OR  |x| > 5
# -------------------------------------------------------------
mask = (ratio.abs() < 0.2) | (ratio.abs() > 5)

# A. 0/1 flag
df["check"] = mask.any(axis=1).astype("int8")

# B. (optional) list the culprit columns, makes debugging trivial
df["check_cols"] = mask.apply(
    lambda row: ", ".join(ratio_cols[row.values]) if row.any() else "", axis=1
)

# -------------------------------------------------------------
# 5. Save or inspect
# -------------------------------------------------------------
df.to_csv("chainlinked_govdef_GDP_flagged.csv", index=False)

# Show only rows that failed, together with the reason
print(
    df.loc[df["check"] == 1, ["ISO3", "year", "check", "check_cols"]]
)

TypeError: only integer scalar arrays can be converted to a scalar index

In [33]:
import pandas as pd
import numpy as np


def flag_outliers(
    path_in: str = "chainlinked_govdef_GDP.dta",
    path_out: str | None = "chainlinked_govdef_GDP_flagged.csv",
) -> pd.DataFrame:
    """
    Read the Stata file, flag rows whose ratios are outside [0.2, 5],
    store the culprit columns, and optionally write a CSV.

    Returns the flagged DataFrame.
    """
    # --------------------------------------------------------- #
    # 1. Always start from a FRESH copy of the data on disk      #
    # --------------------------------------------------------- #
    df = pd.read_stata(path_in)

    # --------------------------------------------------------- #
    # 2. Columns whose names end with '_govdef_GDP' (except the  #
    #    denominator itself) are the ones we want to test        #
    # --------------------------------------------------------- #
    ratio_cols = [
        c for c in df.columns if c.endswith("_govdef_GDP") and c != "govdef_GDP"
    ]

    # --------------------------------------------------------- #
    # 3. Compute the ratios in a SEPARATE object                 #
    # --------------------------------------------------------- #
    ratio = df[ratio_cols].div(df["govdef_GDP"], axis=0)

    # --------------------------------------------------------- #
    # 4. Out-of-band if |x| < 0.2 or |x| > 5                     #
    # --------------------------------------------------------- #
    mask = (ratio.abs() < 0.2) | (ratio.abs() > 5)

    # A) 0 / 1 flag
    df["check"] = mask.any(axis=1).astype("int8")

    # B) names of the columns that triggered the flag
    #    (row.index is a Pandas Index → Boolean masking works)
    df["check_cols"] = mask.apply(
        lambda r: ", ".join(r.index[r]) if r.any() else "", axis=1
    )

    # --------------------------------------------------------- #
    # 5. Persist, if requested                                   #
    # --------------------------------------------------------- #
    if path_out is not None:
        df.to_csv(path_out, index=False)

    return df


# ------------------------------------------------------------------
# Example: run the function and inspect the failures
# ------------------------------------------------------------------
if __name__ == "__main__":
    flagged = flag_outliers()

    # Show only the “problem” rows and why they failed
    cols_to_show = ["ISO3", "year", "check", "check_cols"]
    print(flagged.loc[flagged["check"] == 1, cols_to_show])

      ISO3    year  check                               check_cols
10     ZWE  2019.0      1                          AFDB_govdef_GDP
17     ZWE  2012.0      1                          AFDB_govdef_GDP
129    ZMB  2006.0      1                          AFDB_govdef_GDP
180    ZAF  2005.0      1  Mitchell_govdef_GDP, OECD_EO_govdef_GDP
196    ZAF  1989.0      1                           CS1_govdef_GDP
...    ...     ...    ...                                      ...
14408  AGO  2004.0      1                       IMF_GFS_govdef_GDP
14412  AGO  2000.0      1                       IMF_GFS_govdef_GDP
14415  AGO  1997.0      1                          AFDB_govdef_GDP
14443  AFG  2013.0      1                           ADB_govdef_GDP
14447  AFG  2009.0      1       ADB_govdef_GDP, IMF_GFS_govdef_GDP

[1627 rows x 4 columns]
